In [15]:
using LinearAlgebra

In [16]:
function upper_hessenberg_form(A)
  n = size(A, 1)
  Q = Array{Float64}(I, (n, n))
  for k = 1:n-2
    beta = maximum(abs.(A[k+1:n, k]))
    if beta != 0
      ### Setting up Qk and Q
      A[k+1:n, k] /= beta
      tao = norm(A[k+1:n, k], 2) # norm() is the euclidean norm
      if A[k+1, k] < 0 
        tao = - tao
      end
      eta = A[k+1, k] + tao
      A[k+1, k] = 1
      A[k+2:n, k] /= eta
      gamma = eta/tao
      tao *= beta
      IDN = Array{Float64}(I, (k, k))
      N = zeros(k, n-k)
      v = A[k+1:n, k]
      Qk = Array{Float64}(I, (n-k, n-k)) - gamma * (v * v')
      P = [IDN N; N' Qk]
      Q *= P
      ###
       
      ### Multiply on the left by Qk
      b = transpose(A[k+1:n, k]) * A[k+1:n, k+1:n]
      b *= - gamma
      A[k+1:n, k+1:n] += (A[k+1:n, k] * b)  
      ###

      ### Multiply on the right by Qk
      b = A[:, k+1:n] * A[k+1:n, k]
      b *= -gamma 
      A[:, k+1:n] += (b * transpose(A[k+1:n, k]))
      ###   

      A[k+1,k] = - tao
      A[k+2:n,k] = zeros(n-k-1)
    end
  end
  return A, Q
end

upper_hessenberg_form (generic function with 1 method)

In [17]:
# test H e Q corretas
# A = rand(Float64, (4,4))
# display(A)
# B = hessenberg(A)
# H, Q = upper_hessenberg_form(A)
# display(B)
# display(H)
# display(Q'Q)
# display(Q*H*Q')

In [35]:
# 2.3.1 Autovalores de uma matriz 2x2
function eigva22(A)
  s = sum((abs(element) for element in A))
  A *= 1/s

  # (A - tI) tr e det
  t = tr(A)/2
  C = A - t * Array{Float64}(I, (2,2))
  # display(C)
  # display(tr(C))
  d = ((A[1,1] - t) * (A[2,2] - t)) - (A[1,2] * A[2,1])
  # display(d)

  eigva = [t + sqrt(abs(d)), t - sqrt(abs(d))]
  if d > 0
    eigva = s * [t + (1im * sqrt(d)), t - (1im * sqrt(d))]
  end

  # checando os autovalores
#   display(eigen(A).values)
#   display(eigva)
  return eigva
end

eigva22 (generic function with 1 method)

In [110]:
function get_shifts(H)
  H_canto = H[n-1:n, n-1:n]
  shifts = eigva22(H_canto)
  if shifts isa Array{Float64,1}
  #   display(shifts)
    dif = abs.(shifts - [H[n,n], H[n,n]])
    s_w = shifts[argmin(dif)]
    shifts = [s_w, s_w]
  #   display(shifts)
  end
  # display(H[n,n])
end


2-element Array{Float64,1}:
  0.3467307152175203 
 -0.47379856388501584

2-element Array{Float64,1}:
 0.3467307152175203
 0.3467307152175203

0.21157538950970786

In [125]:
n = 5
A = rand(Float64, (n,n))
H, Q = upper_hessenberg_form(A)

### calculando p
first_line = ((H[1,1] - shifts[1]) * (H[1,1] - shifts[2])) + (H[1,2]*H[2,1])
second_line = H[2,1] * ((H[1,1] + H[2,2]) - (shifts[1] + shifts[2]))
third_line = H[3,2] * H[2,1]
remaining = zeros(n-3)
p = [first_line; second_line; third_line; remaining]
display(p)
###

# checking
# Inn = Array{Float64}(I, (n,n))
# p2 = (H - (shifts[1] * Inn)) * (H - (shifts[2] * Inn))
# p2[1:n,1]


5-element Array{Float64,1}:
  1.107187253353125 
 -2.292143952270201 
 -0.8802992609653799
  0.0               
  0.0               

5-element Array{Float64,1}:
  1.107187253353125 
 -2.2921439522702016
 -0.8802992609653799
  0.0               
  0.0               